<a href="https://colab.research.google.com/github/epsilon-deltta/sl-for-dl/blob/master/dl/test-lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ml

#### sonar 

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy
import pandas as pd
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

df = pd.read_csv('../dataset/sonar.csv', header=None)

df.info()

# dataset = df.values
X = df.iloc[:,0:60]
Y_obj = df.iloc[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
# 빈 accuracy 배열
accuracy = []
X = X.to_numpy()

In [ ]:
from sklearn.model_selection import StratifiedKFold
# 10개의 파일로 쪼갬
n_fold = 2
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

model.save('sonar_model.h5')

from keras.models import Sequential, load_model

exsonar = load_model('sonar_model.h5')

#### iris

In [44]:
# iris
from sklearn.datasets import load_iris
iris = load_iris()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)

# iris preprocessing
from sklearn.datasets import load_iris
iris = load_iris()
from sklearn.model_selection import train_test_split

df = pd.DataFrame(data=iris.data,columns=iris.feature_names)

## Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df = sc.fit_transform(df)
df = pd.DataFrame(df,columns=iris.feature_names)

df['target'] = iris.target

# OnehotEncoder x : lightGBM 
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder()
# target = enc.fit_transform(iris.target.reshape(-1,1)).toarray()
print(df['target'].shape)
xtr,xte, ytr,yte = train_test_split(df.iloc[:,:-1], df['target'])

(150,)


In [53]:
def lgb_evaluate(numLeaves, maxDepth, scaleWeight, minChildWeight, subsample, colSam, output = 'score'):
    reg=lgb.LGBMClassifier(num_leaves=31, max_depth= 2,scale_pos_weight= scaleWeight, min_child_weight= minChildWeight, subsample= 0.4, colsample_bytree= 0.4, learning_rate=0.05,   n_estimators=20)
    scores = cross_val_score(reg, xtr, ytr, cv=5, scoring='accuracy') # cross_val_score
    # scores = cross_val_score(reg, train_x, train_y, cv=5, scoring='neg_mean_squared_error')
    
    if output == 'score' :
      return np.mean(scores)
    if output == 'model' :
      return reg

def bayesOpt(xtr, ytr):
    lgbBO = BayesianOptimization(lgb_evaluate, {  'numLeaves':  (5, 90),  'maxDepth': (2, 90),   'scaleWeight': (1, 10000),  'minChildWeight': (0.01, 70), 'subsample': (0.4, 1), 'colSam': (0.4, 1) })
    lgbBO.maximize(init_points=5, n_iter=30)
    print(lgbBO.res)
    return lgbBO

lgbBO = bayesOpt(xtr, ytr)

# 최적 파라메터 저장 
params = lgbBO.max['params']
 
# 모델에 적용
model = lgb_evaluate(
    numLeaves = params['numLeaves'],
     maxDepth = params['maxDepth'],
     scaleWeight = params['maxDepth'],
     minChildWeight = params['minChildWeight'],
     subsample = params['subsample'],
     colSam = params['colSam'],
     output = 'model'
     )

|   iter    |  target   |  colSam   | maxDepth  | minChi... | numLeaves | scaleW... | subsample |
-------------------------------------------------------------------------------------------------
|  1        |  0.3664   |  0.8884   |  81.73    |  56.97    |  74.26    |  500.9    |  0.9632   |
|  2        |  0.3664   |  0.8693   |  69.57    |  23.59    |  71.83    |  9.401e+0 |  0.5525   |
|  3        |  0.3664   |  0.9294   |  73.63    |  52.11    |  71.83    |  2.862e+0 |  0.5948   |
|  4        |  0.3664   |  0.4297   |  9.446    |  46.3     |  64.94    |  5e+03    |  0.7071   |
|  5        |  0.3664   |  0.5744   |  71.58    |  59.67    |  10.59    |  2.716e+0 |  0.4401   |
|  6        |  0.3573   |  0.4213   |  40.36    |  20.4     |  8.144    |  1e+03    |  0.4265   |
|  7        |  0.3664   |  0.469    |  89.64    |  67.47    |  7.808    |  7.572e+0 |  0.7674   |
|  8        |  0.3664   |  0.7361   |  78.73    |  66.73    |  8.138    |  9.957e+0 |  0.7313   |
|  9        |  0.366

#### mpg (reg)

In [ ]:
import seaborn as sns 
df = sns.load_dataset('mpg') # iris 

df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 
# horsepower 열의 자료형 변경 (문자열 ->숫자)
df['horsepower'].replace('?', np.nan, inplace=True)      # '?'을 np.nan으로 변경
df.dropna(subset=['horsepower'], axis=0, inplace=True)   # 누락데이터 행을 삭제
df['horsepower'] = df['horsepower'].astype('float')      # 문자열을 실수형으로 변환
# 분석에 활용할 열(속성)을 선택 (연비, 실린더, 출력, 중량)
ndf = df[['mpg', 'cylinders', 'horsepower', 'weight']]

X=ndf[['cylinders', 'horsepower', 'weight']]  #독립 변수 X1, X2, X3
y=ndf['mpg']     #종속 변수 Y
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

from sklearn.linear_model import LinearRegression
lr = LinearRegression()   
lr.fit(X_train, y_train)

r_square = lr.score(X_test, y_test)
print(r_square) #  결정계수(R-제곱) 계산
print('X 변수의 계수 a: ', lr.coef_) # 회귀식의 기울기
print('상수항 b', lr.intercept_) # 회귀식의 y절편

# train data의 산점도와 test data로 예측한 회귀선을 그래프로 출력 
y_hat = lr.predict(X_test)
plt.figure(figsize=(10, 5))
ax1 = sns.distplot(y_test, hist=False, label="y_test")
ax2 = sns.distplot(y_hat, hist=False, label="y_hat", ax=ax1)
plt.show()
plt.close()

#### breast_cancel (class)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from lightgbm import plot_importance
import matplotlib.pyplot as plt
import Common_Module.CMStat as CO

dataset = load_breast_cancer()
ftr = dataset.data
target = dataset.target

X_train, X_test, y_train, y_test = train_test_split(ftr, target, test_size=0.2, random_state=156)


In [ ]:
from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(n_estimators=400)
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
pred = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:1]
CO.get_clf_eval(y_test, pred)

fig, ax = plt.subplots(figsize=(10,12))
plot_importance(lgbm_wrapper, ax=ax)
plt.show()
# 평가지표 출력하는 함수 설정
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred, pred_proba)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

### vision 

#### cifar-10 

In [ ]:
# classification 

### nlp

#### imdb  

In [ ]:
# -*- coding: utf-8 -*-
# 코드 내부에 한글을 사용가능 하게 해주는 부분입니다.

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)
(x_train, ytr), (x_test, yte) = imdb.load_data(num_words=5000)
# x_train = x_train[:12500]
# x_test = x_test[:12500]
# ytr = ytr[:12500]
# yte = yte[:12500]
xtr = sequence.pad_sequences(x_train, maxlen=50)
xte = sequence.pad_sequences(x_test, maxlen=50)

In [ ]:
# 모델의 설정
model = Sequential()
model.add(Embedding(5000, 100 )   )
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# model.summary()

inputA = Input(shape=(50,) )
x = Embedding(5000,50)(inputA)
x = Dropout(0.5)(x)
x = Conv1D(64, 5, padding='valid', activation='relu',strides=1)(x)
x = MaxPooling1D(pool_size=4)(x)
x = LSTM(55)(x)
x = Dense(1,activation='sigmoid')(x)
model1 = Model(inputA,x)


import datetime , tensorflow as tf
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# 모델의 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델의 실행
history = model.fit(xtr, ytr, batch_size=250, epochs=1, validation_data=(xte, yte),callbacks=[tensorboard_callback])


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()